## Importsetup

In [2]:
from app.main.extraction.dxf_extraction_coordinator import DxfExtractionCoordinator
from app.main.models.wall import Wall 
from app.main.models.architecture_wall import Architecture_Wall
from app.main.models.point import Point 
from app.main.models.window import Window
from app.main.models.line import Line
from app.main.services.walls_extraction import WallsExtraction
from app.main.services.dxf_extraction_simple import DxfExtractionSimple
from app.main.dtos.request.extraction_request_file import ExtractionRequestFile
from app.main.models.line import Line 
from app.main.constants import (PLOT_DPI, PLOT_LINE_WIDTH_LARGE,
                                PLOT_LINE_WIDTH_SMALL, DefaultColors,
                                OrientationColors, PlotStyle, PlotTypes)
from app.main.models.facade_arc import FacadeArc
###########################################################################
from shapely.geometry.polygon import LinearRing
from shapely.geometry.polygon import Polygon as shapePolygon
from shapely.geometry import Point as shapePoint
from shapely.geometry import LineString
from shapely.geometry import Point as ShapelyPoint
import math
import numpy as np
import itertools as it
from scipy.spatial.distance import cdist
import ezdxf
import uuid

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import operator

###########################################################################
file_path_simple = '../../media/Einfache_Plaene_Compass/'
file_path_architecture = '../../media/Architektenplaene/'
###########################################################################
file_name_simple = 'R-Bau_D_EG.dxf'
file_name_architecutre = 'R-Bau_A_EG.dxf'
###########################################################################
simple_dxf = ezdxf.readfile(file_path_simple+file_name_simple)
architecture_dxf = ezdxf.readfile(file_path_architecture+file_name_architecutre)
simple_model = simple_dxf.modelspace()
arichtecture_model = architecture_dxf.modelspace()

## In the block below there will be created a dxf-File for analysis purposes.

In [13]:
#Read pillar circles from architecture wall
architecture_polylines = []
architecture_lines = []
fassaden_polylines = arichtecture_model.query("LWPolyline[layer=='A_05_FASSADE_ELEMENTE']")#.first.get_points('xy')

#lines
fassaden_lines = arichtecture_model.query("LINE[layer=='A_05_FASSADE_ELEMENTE']")

#Arcs
fassaden_arc = arichtecture_model.query("ARC[layer=='A_05_FASSADE_ELEMENTE']")

for polyline in fassaden_polylines:
    architecture_polylines.append(polyline)
for line in fassaden_lines:
    architecture_lines.append(line)

doc = ezdxf.new('R2000')
msp = doc.modelspace()
doc.layers.new(name='FASSADE_LINES', dxfattribs={'linetype': 'DASHED', 'color': 1})
doc.layers.new(name='FASSADE_POLYLINES', dxfattribs={'linetype': 'DASHED', 'color': 2})
doc.layers.new(name='FASSADE_ARCS', dxfattribs={'linetype': 'DASHED', 'color': 3})

#linetypes = ['BYLAYER', 'HIDDEN', '4.102_EG$0$CONTINOUS']
linetypes = ['HIDDEN']
for x in architecture_polylines:
    #mylinetype = x.dxf.linetype
    #if mylinetype not in linetypes:
    #    linetypes.append(mylinetype)
    if x.dxf.linetype in linetypes:
        msp.add_lwpolyline(x, dxfattribs={'layer': 'FASSADE_POLYLINES'})

#add lines to drawing
coordinates = []

for e in architecture_lines:
    #mylinetype = e.dxf.linetype
    #if mylinetype not in linetypes:
    #    linetypes.append(mylinetype)
    if e.dxftype() == 'LINE' and e.dxf.linetype in linetypes:
        tupel = (e.dxf.start, e.dxf.end)
        coordinates.append(tupel)

for x in coordinates:
    msp.add_line(x[0], x[1], dxfattribs={'layer': 'FASSADE_LINES'})


#add arcs to drwawing
for x in fassaden_arc:
    #mylinetype = x.dxf.linetype
    #if mylinetype not in linetypes:
    #    linetypes.append(mylinetype)
    if  x.dxf.linetype in linetypes:
        msp.add_arc(x.dxf.center, x.dxf.radius, x.dxf.start_angle, x.dxf.end_angle, True, dxfattribs={'layer': 'FASSADE_ARCS'})

print(linetypes)
doc.saveas("facade_arcs.dxf")


['HIDDEN']


In [19]:
'''This method creates a custom dxf file with all lines an arcs of an list of arcs. '''
def draw_arcs_and_lines(facade_arcs: list[FacadeArc]):
    file_name = "facade_arcs_and_lines.dxf"
    doc = ezdxf.new('R2000')
    msp = doc.modelspace()  
    doc.layers.new(name='FASSADE_ARCS', dxfattribs={'linetype': 'DASHED', 'color': 3})
    doc.layers.new(name='FASSADE_ARC_LINES', dxfattribs={'linetype': 'DASHED', 'color': 1})
    for facade_arc in facade_arcs:
        center_point = convert_point_to_coordinate(facade_arc.center_point)
        msp.add_arc(center_point, facade_arc.radius, facade_arc.start_angle, facade_arc.end_angle, True, dxfattribs={'layer': 'FASSADE_ARCS'})
        if not facade_arc.lines:
            continue
        for line in facade_arc.lines:
            start_point = convert_point_to_coordinate(line.start_point)
            end_point = convert_point_to_coordinate(line.end_point)
            msp.add_line(start_point, end_point, dxfattribs={'layer': 'FASSADE_ARC_LINES'})
    doc.saveas(file_name)


''' This method converts a point object int a coordinate. '''    
def convert_point_to_coordinate(point: Point):
    return (point.x_coordinate, point.y_coordinate)


In [20]:
''' This method creates a point object from an coordinate tuple.'''
def create_point_from_coordinate(coordinate: tuple) -> Point:
    return Point(x_coordinate=coordinate[0], y_coordinate=coordinate[1])

In [21]:
#quick check
np.sin(np.deg2rad(90))
np.cos(np.deg2rad(90))

6.123233995736766e-17

In [22]:
''' This method creates a linestring from two point objects.'''
def create_line_string_of_two_points(point_1: Point, point_2: Point)-> LineString:
    return LineString([(point_1.x_coordinate, point_1.y_coordinate), (point_2.x_coordinate, point_2.y_coordinate)])

In [23]:
''' This method reads all the arc ezdxf entities from the layer fassaden_elemente and transforms them to facade_arcs'''
def read_arcs(dxf_file) -> list[FacadeArc]:
    architecture_model = architecture_dxf.modelspace()
    dxf_arcs = architecture_model.query("ARC[layer=='A_05_FASSADE_ELEMENTE']")
    facade_arcs = []
    for dxf_arc in dxf_arcs:
        if dxf_arc.dxf.radius > 2.0:
            facade_arcs.append(convert_dxf_to_object_arc(dxf_arc))
    return facade_arcs

''' This method converts a edxf arc into a facadearc object. '''
def convert_dxf_to_object_arc(dxf_arc) -> FacadeArc:
    center_point = create_point_from_coordinate(dxf_arc.dxf.center)
    start_point = create_point_from_coordinate(dxf_arc.start_point)
    end_point = create_point_from_coordinate(dxf_arc.end_point)

    radius = dxf_arc.dxf.radius
    start_angle = dxf_arc.dxf.start_angle
    end_angle = dxf_arc.dxf.end_angle

    perimeter = 2 * math.pi * radius * ((end_angle - start_angle) / 360)
    delta_angle = get_delta_angle(perimeter, start_angle, end_angle)
    lines = get_lines(start_angle, end_angle, delta_angle, center_point, start_point, end_point, radius)

    return FacadeArc(center_point, start_point, end_point, radius, start_angle, end_angle, perimeter=perimeter, delta_angle=delta_angle, lines=lines)

''' This method gets the lenght per line.'''
def get_line_length(perimeter: float):
    MIN_LINE_LENGTH = 5
    if perimeter <= MIN_LINE_LENGTH:
        return perimeter
    
    ratio = perimeter / MIN_LINE_LENGTH
    amount_lines = math.floor(ratio)
    return perimeter / amount_lines

''' This method calculates the delta angle for detecting shared lenghts. '''  
def get_delta_angle(perimeter: float, start_angle: float, end_angle: float):
    line_length = get_line_length(perimeter)
    amount_lines = perimeter / line_length
    return (end_angle - start_angle) / amount_lines

''' This method creates a list of lines for one given arc.'''
def get_lines(start_angle: float, end_angle: float, delta_angle: float, center_point: Point, start_point: Point, end_point: Point, radius: float):

    first_point = end_point
    first_angle = end_angle
    line_count = int((end_angle - start_angle) / delta_angle)
    lines = []
    for i in range(line_count):
        second_angle = first_angle - (i + 1) * delta_angle
        second_point = get_arc_point(center_point, radius, second_angle)

        line = create_line_from_two_points(first_point, second_point)
        lines.append(line)
        first_point = second_point

    return lines

''' This method creates a Point object on a radius line segment.'''
def get_arc_point(center_point: Point, radius: float, angle: float) -> Point:
    dx = radius * np.cos(np.deg2rad(angle))
    dy = radius * np.sin(np.deg2rad(angle))

    x_coordinate = center_point.x_coordinate + dx
    y_coordinate = center_point.y_coordinate + dy
    return Point(x_coordinate, y_coordinate)

''' This method cretes a line object form two point objects.'''
def create_line_from_two_points(start_point: Point, end_point: Point):
    line_string = create_line_string_of_two_points(start_point, end_point)
    return Line("id", line_string, start_point=start_point, end_point=end_point)

''' This method creates a Linestring from two point objects. '''
def create_line_string_of_two_points(point_1: Point, point_2: Point)-> LineString:
    return LineString([(point_1.x_coordinate, point_1.y_coordinate), (point_2.x_coordinate, point_2.y_coordinate)])

'''
facade_arcs = read_arcs(architecture_dxf)
for facade_arc in facade_arcs:
    print("###################")
    print(facade_arc)
draw_arcs_and_lines(facade_arcs)
'''

'''
arcs = read_arcs(architecture_dxf)
draw_arcs_and_lines(arcs)
#print(arcs)
data = []
for arc in arcs:
    data.append(arc.perimeter)

print(data)'''
print(get_line_length(0.4))

0.4


In [11]:
''' This method plots a lines of an arcs list.'''
def plot_lines_of_arcs(facade_arcs: list[FacadeArc]):
    for facade_arc in facade_arcs:
        for line in facade_arc.lines:
            plt.plot([line.start_point.x_coordinate, line.end_point.x_coordinate], [line.start_point.y_coordinate, line.end_point.y_coordinate], color="b", linewidth=PLOT_LINE_WIDTH_LARGE)
            plt.savefig("facade_plot.png", dpi=PLOT_DPI)
            plt.clf()

'''This method creates a line object form two given coordinates.'''
def get_line_from_coordinates(coordinate_1, coordinate_2):
    line_string = get_line_string_of_two_coordinates(coordinate_1, coordinate_2)
    start_point = create_point_from_coordinate(coordinate_1)
    end_point = create_point_from_coordinate(coordinate_2)
    return Line("id", line_string, start_point=start_point, end_point=end_point)

''' This method creates a list of Line objects from a polyline'''
def create_lines_from_polylines(polylines: list) ->list[Line]:
    lines = []
    for polyline in polylines:
        for i in range(len(polyline)-1):
            line = get_line_from_coordinates(polyline[i], polyline[i + 1])
            lines.append(line)
    return lines


''' This method creates a Linestring from two coordinates. '''
def get_line_string_of_two_coordinates(coordinate_1, coordinate_2):
    return LineString([(coordinate_1[0], coordinate_1[1]), (coordinate_2[0], coordinate_2[1])])

In [24]:
''' This method gets the lenght between two points.'''
def get_length_between_points(point_1: Point, point_2: Point) -> float:
    line_string = LineString([(point_1.x_coordinate, point_1.y_coordinate), (point_2.x_coordinate, point_2.y_coordinate)])
    return line_string.length

In [25]:
''' This method creates a linestring form two coordinates.'''
def get_line_string_of_two_coordinates(coordinate_1, coordinate_2):
    return LineString([(coordinate_1[0], coordinate_1[1]), (coordinate_2[0], coordinate_2[1])])

In [26]:
''' This method creates a Line object from two coordinates.'''
def get_line_from_coordinates(coordinate_1, coordinate_2):
    line_string = get_line_string_of_two_coordinates(coordinate_1, coordinate_2)
    start_point = create_point_from_coordinate(coordinate_1)
    end_point = create_point_from_coordinate(coordinate_2)
    return Line("id", line_string, start_point=start_point, end_point=end_point)

''' This method creates a list of lines from a polyline. '''
def create_lines_from_polylines(polylines: list) ->list[Line]:
    lines = []
    for polyline in polylines:
        for i in range(len(polyline)-1):
            line = get_line_from_coordinates(polyline[i], polyline[i + 1])
            lines.append(line)
    return lines

''' This method creates a linestring from a wall object.'''
def create_line_string_for_wall(wall: Wall):
    return LineString([(wall.start_point.x_coordinate, wall.start_point.y_coordinate), (wall.end_point.x_coordinate, wall.end_point.y_coordinate)])

''' This method checks if a point is on a given linestring including a threshold value.'''
def is_point_on_line(point: Point, line: LineString):
    threshold = 0.05
    coordinate = ShapelyPoint(point.x_coordinate, point.y_coordinate)
    return line.distance(coordinate) < threshold
    

### The following block is for testing purposes.

In [27]:
facade_polylines = arichtecture_model.query("LWPolyline[layer=='A_05_FASSADE_ELEMENTE']")#.first.get_points('xy')
lines = create_lines_from_polylines(facade_polylines)
#lines
facade_lines = arichtecture_model.query("LINE[layer=='A_05_FASSADE_ELEMENTE']")
for facade_line in facade_lines:
    line = get_line_from_coordinates(facade_line.dxf.start, facade_line.dxf.end)
    lines.append(line)

arcs = read_arcs(architecture_dxf)
print(len(lines))
amount_arc_lines = 0
for arc in arcs:
    amount_arc_lines += len(arc.lines)
print(amount_arc_lines)
#plot_lines_of_arcs(arcs)

5188
198


### The follwoing block is for extracting a simple plan for testing purposes.

In [28]:
#Simple plan auslesen
dxf_file = ezdxf.readfile(file_path_simple + file_name_simple)
dxf_extraction_simple = DxfExtractionSimple()
simple_polygons, polygons_with_virtual_entities, extracted_functionalities, counter_polygon_closed = dxf_extraction_simple.get_all_polygons(dxf_file ,file_path_simple + file_name_simple)


2021-06-18 11:51:35.747 | SUCCESS  | app.main.services.orientation_extraction:get_north_orientation:116 - Compass extraction successful, return coordinate system
2021-06-18 11:51:35.747 | INFO     | app.main.services.dxf_extraction_simple:get_all_polygons:97 - Orientation will be calculated with compass.
2021-06-18 11:51:35.748 | SUCCESS  | app.main.services.extraction_helper:get_modelspace:191 - sucessfully get_modelspace() from dxf_file: <ezdxf.document.Drawing object at 0x0000027A3C0DC160>
2021-06-18 11:51:35.749 | DEBUG    | app.main.services.extraction_helper:get_outline_polygon:200 - get_outline_polygon for modelspace: <ezdxf.layouts.layout.Modelspace object at 0x0000027A3B1E2B50>
2021-06-18 11:51:35.794 | DEBUG    | app.main.services.dxf_extraction_simple:get_all_polygons:145 - got all room_polylines on layer: RAUMPOLYGON
2021-06-18 11:51:36.152 | INFO     | app.main.services.dxf_extraction_simple:get_all_polygons:192 - Extracting neighbours...
Neighbours - Progress: 100%|██████

In [29]:
''' This method calculates the window lenght for a wall depending on a list of arcs and lines.'''
def calculate_window_length(wall: Wall, facade_arcs: list[FacadeArc], lines: list[Line]) -> float:
    window_length = get_window_length_of_wall(wall, lines)
    window_length_sum = 0.0
    wall_length = get_length_between_points(wall.start_point, wall.end_point)
    if window_length < wall_length:
        for facade_arc in facade_arcs:
            if facade_arc.lines:
                window_length_new = get_window_length_of_wall(wall, facade_arc.lines)
                window_length_sum += window_length_new
    return min(window_length + window_length_sum, wall_length)

In [30]:
''' This method gets the window lenght from an given wall.'''
def get_window_length_of_wall(wall: Wall, lines: list[Line])->float:
    wall_line_string = create_line_string_for_wall(wall)
    window_length_sum = 0.0
    for line in lines:
        window_length = 0.0
        if not is_point_on_line(wall.start_point, line.line_string) and not is_point_on_line(wall.end_point, line.line_string):
            if is_point_on_line(line.start_point, wall_line_string) and is_point_on_line(line.end_point, wall_line_string):
                window_length = get_length_between_points(line.start_point, line.end_point)
                #print("Länge berechnet! Option 1")
           
        elif is_point_on_line(wall.start_point, line.line_string) and is_point_on_line(wall.end_point, line.line_string):
            window_length = get_length_between_points(wall.start_point, wall.end_point)
            #print("Länge berechnet! Option 2")
        else:
            wall_point_on_line = None
            line_point_on_wall = None
            if is_point_on_line(wall.start_point, line.line_string):
                wall_point_on_line = wall.start_point
            elif is_point_on_line(wall.end_point, line.line_string):
                wall_point_on_line = wall.end_point
        
            if is_point_on_line(line.start_point, wall_line_string):
                line_point_on_wall = line.start_point
            elif is_point_on_line(line.end_point, wall_line_string):
                line_point_on_wall = line.end_point
            
            if wall_point_on_line and line_point_on_wall:
                window_length = get_length_between_points(wall_point_on_line, line_point_on_wall)
                #print("Länge berechnet! Option 3")
        
        window_length_sum += window_length
    return window_length_sum

### The following block is used for testing the logic above.

In [32]:
facade_polylines = arichtecture_model.query("LWPolyline[layer=='A_05_FASSADE_ELEMENTE']")#.first.get_points('xy')
lines = create_lines_from_polylines(facade_polylines)
#lines
facade_lines = arichtecture_model.query("LINE[layer=='A_05_FASSADE_ELEMENTE']")
for facade_line in facade_lines:
    line = get_line_from_coordinates(facade_line.dxf.start, facade_line.dxf.end)
    lines.append(line)

print("Amount of lines found in facade layer: {}".format(str(len(lines))))

amount_walls = 0
for polygon in simple_polygons:
    for wall in polygon.walls:
        if not wall.is_outer_wall:
            continue
        amount_walls += 1

print("Amount of walls found in simple plan: {}".format(str(amount_walls)))

facade_arcs = read_arcs(architecture_dxf)

window_lengths = []
counter = 0
for polygon in simple_polygons:
    for wall in polygon.walls:
        if not wall.is_outer_wall:
            continue
        progress = round((counter / amount_walls) * 100, 2)
        if progress % 5 == 0:
            print("Progress: {} %".format(str(progress)))
        window_length = calculate_window_length(wall, facade_arcs, lines)
        wall.window_length = window_length
        window_lengths.append(window_length)
        counter += 1

print(window_lengths)
#minimale window length, maximale window length, wieviele haben length

Amount of lines found in facade layer: 5188
Amount of walls found in simple plan: 329
Progress: 0.0 %
[6.439507147243732, 6.439507147243734, 0.001410583207413587, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.430343020278245, 0.0, 0.0, 1.8860686040556922, 6.244299935442258, 0.0, 0.0, 1.2488599870884638, 3.764279138697746, 3.764336383744076, 0.08421361158681713, 0.07860841079821038, 0.0, 6.374243325710002, 5.844456216908312, 0.0, 1.7902576310894016, 0.0, 2.429997999975674, 0.10356676813103927, 0.0, 7.390261203383382, 1.4780522406766796, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.407348606509522, 0.0, 0.0, 1.4814697213019041, 0.0, 2.355000185342913, 7.663890312032113, 4.960000000000001, 3.5600000000000005, 3.660000000000002, 1.9340631779610493, 9.670315889805234, 0.39224999383894205, 3.5424999999967994, 0.0, 3.5249999999999986, 0.0, 0.0, 3.549999999999997, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.556359795276089, 0.04463184874666115, 0.0, 0.0, 0.47999999999998977, 6.

### The following block creates a dxf file for testing purposes.

In [29]:
#write simple plan to architecture plan (just polygons)
doc = architecture_dxf
new_msp = doc.modelspace()
doc.layers.new(name='SIMPLE_PLAN', dxfattribs={'linetype': 'DASHED', 'color': 10})
polylines = simple_model.query(
            'LWPolyline[layer=="{}"]'.format("RAUMPOLYGON"))
for polyline in polylines:
    new_msp.add_lwpolyline(polyline, dxfattribs={'layer': 'RAUMPOLYGON'})
doc.saveas("architecture_simple.dxf")